In [1]:
import os
os.chdir("C:\\Users\\akadali\\Desktop\\Deep_NLP\\MLG_Capstone_ChatBot\\Chatbot_Intent_Classification - Glove")

In [2]:
import json
intents = []
questions = []
words = []
file = open("train_data2.json").read()
qa_file = json.loads(file)

In [3]:
for q in qa_file['intents']:
    if q['tag'] not in intents:
        intents.append(q['tag'])
        for pattern in q['patterns']:
            questions.append(pattern)

In [4]:
import nltk
import numpy as np
import pandas as pd
from nltk import word_tokenize

In [5]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_model_300d.h5')

### Getting Embeddings from Glove

In [6]:
# Getting embeddings from Glove
os.chdir("C:/Users/akadali/Desktop/Deep_NLP/MLG_Capstone_ChatBot/Chatbot_Intent_Classification - Glove")
embeddings_index = dict()
f = open('glove.6B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [7]:
# Creating a feature vector by averaging embeddings for all words in a sentence with Glovebox embeddings
def embedding_feats_300(query):
    tokens = word_tokenize(query)
    DIMENSION = 300
    zero_vector = np.zeros(DIMENSION)
    #feats = []
    feat_for_this = zero_vector
    count_for_this = 0
    for token in tokens:
        if token in embeddings_index:
            feat_for_this += embeddings_index[token][:300]
            count_for_this+= 1
    feats = feat_for_this/count_for_this
    return feats

In [12]:
import random
question = "How can I submit my referral"
feat = embedding_feats_300(question)
result_index = np.argmax(model.predict(feat.reshape(1,-1)))
result = intents[result_index]
for q in qa_file['intents']:
    if q['tag'] == result:
        response = random.choice(q['responses'])
        print(response)
        break

My pleasure. Happy to help!


In [21]:
data['answer'].iloc[10]

'If you are no longer an active employee/leaving the firm before your referral completes 30days with the firm, you will be eligible for the Alumni Referral Program. The alumni Referral Program payment can be paid out by Payroll if you were employed within the current calendar year or by Accounts Payable if you were not employed in the current calendar year. \n\nYou will find more details in the webpage here - https://www2.deloitte.com/us/en/pages/careers/articles/alumni-referral-deloitte-us-join-deloitte.html'

## Estalishing connection to the 'Talent referral payout' database in SQL
* Establish connection with the TRP databases (both payout and referral databases) in SQL Workbench
* Write functions to retrieve the data based on user request.

In [ ]:
import mysql.connector
mydb = mysql.connector.connect(host="localhost", 
                               user="root", 
                               passwd="root",
                               auth_plugin = 'mysql_native_password',
                               database = "talent_referral_payout")

In [ ]:
mycursor = mydb.cursor(buffered = True)

In [ ]:
#Definition to obtain te referral bonus payout history of a referring professional

def search_pay_db(emp_id):
    emp_id = str(emp_id)
    #print(type(emp_id))
    query = "SELECT referral_name,amount,date_of_origin FROM trp_payment_report_sample WHERE emp_num = "+ emp_id
    mycursor.execute(query)
    return mycursor.fetchone()

In [ ]:
#Definition to obtain the status of referrals submitted by the referring professional

def search_status_db(rms_id):
    rms_id = str(rms_id)
    #print(type(emp_id))
    query = "SELECT candidate_name,requisition_no,current_status,reference_date FROM trp_status WHERE candidate_id = " + rms_id
    mycursor.execute(query)
    return mycursor.fetchone()

# Chatbot GUI

# Methods for Chatbot GUI response 

In [ ]:
def gui_response(user_input):
    user_input = str(user_input)
    if len(user_input) == 0:
        response = "Looks like you have not asked any question. Please feel free to ask me anything related to referral program."
    else:
        user_input = text_clean(user_input)
        user_input = text_preprocess(user_input)
        q_vec = [embedding_feats_200(user_input)]
        sims = cosine_similarity(q_vec, question_vectors)
        max_s = sims.max()
        if max_s < 0.6:
            response = "I'm sorry, I don't think I understood that correctly, can you please rephrase your question and try again.Feel free to email ustalentreferralprogram@deloitte.com to reach referral team, if I’m not able to solve your problem right now"
        else:
            max_i = np.argmax(sims)
            response = data.answer[max_i]
    return response

In [ ]:
def gui_bonus(emp_id):
    emp_id = str(emp_id)
    ans = search_pay_db(emp_id) 
    if ans is not None:
        res = "You were paid $"+ str(ans[1])+" for referring "+str(ans[0])+" along with "+str(ans[2])+" pay period"
    else:
        res = "Sorry, I couldn't find any result with the input provided" 
    return res

In [ ]:
def gui_status(rms_id):
    rms_id = str(rms_id)
    ans = search_status_db(rms_id)
    if ans is not None:
        res = "Your referral "+str(ans[0])+ " has been "+str(ans[2])+" for requisition with ID:"+str(ans[1])
    else:
        res = "Sorry, I couldn't find any result with the input provided" 
    return res

## Creating Chatbot GUI using tkinter

In [284]:
#Importing tkinter
import tkinter
from tkinter import *

In [285]:
#Enabling High DPI in Windows 10
try:
    from ctypes import windll
    windll.shcore.SetProcessDpiAwareness(1)
except:
    pass

### Methods for Buttons

In [286]:
def send(*args):
    msg = messageWindow.get("1.0",'end-1c')
    messageWindow.delete("0.0",END)
    if msg == '':
        chatWindow.config(state=NORMAL, justify = RIGHT)
        chatWindow.insert(END, "You: \n" + msg + '\n')
        chatWindow.config(foreground="#3b5998", font=("Helvetica Neue", 9, 'bold' ), justify = LEFT)
        res = "Please feel free to ask me anything related to referral program."
        chatWindow.insert(END, "Bot: " + res + '\n\n')
        chatWindow.config(state=DISABLED)
        chatWindow.yview(END)
    else: 
        chatWindow.config(state=NORMAL)
        chatWindow.insert(END, "You: " + msg + '\n')
        chatWindow.config(foreground="#3b5998", font=("Helvetica Neue", 9, 'bold' ))
        res = gui_response(msg)
        chatWindow.insert(END, "Bot: " + res + '\n\n')
        chatWindow.config(state=DISABLED)
        chatWindow.yview(END)

In [287]:
def bonus():
    pers = messageWindow.get("1.0",'end-1c')
    messageWindow.delete("0.0",END)
    if pers == '':
        chatWindow.config(state=NORMAL)
        #chatWindow.insert(END, "You: " + msg + '\n')
        chatWindow.config(foreground="#3b5998", font=("Helvetica Neue", 9, 'bold' ))
        res = "Please enter your Personal Emp.ID and click on 'Bonus History' again"
        chatWindow.insert(END, "Bot: " + res + '\n\n')
        chatWindow.config(state=DISABLED)
        chatWindow.yview(END)
    else:
        chatWindow.config(state=NORMAL)
        chatWindow.insert(END, "Your Emp ID: " + pers + '\n\n')
        chatWindow.config(foreground="#3b5998", font=("Helvetica Neue", 9, 'bold' ))
        res = gui_bonus(pers)
        chatWindow.insert(END, "Bot: " + res + '\n\n')
        chatWindow.config(state=DISABLED)
        chatWindow.yview(END)

In [288]:
def status():
    rms_id = messageWindow.get("1.0",'end-1c')
    messageWindow.delete("0.0",END)
    if rms_id == '':
        chatWindow.config(state=NORMAL)
        #chatWindow.insert(END, "You: " + msg + '\n')
        chatWindow.config(foreground="#3b5998", font=("Helvetica Neue", 9, 'bold' ))
        res = "Please enter your referral's RMS.ID and click on 'Referral Status' again"
        chatWindow.insert(END, "Bot: " + res + '\n\n')
        chatWindow.config(state=DISABLED)
        chatWindow.yview(END)
    else:
        chatWindow.config(state=NORMAL)
        chatWindow.insert(END, "Candidate RMS ID: " + rms_id + '\n\n')
        chatWindow.config(foreground="#3b5998", font=("Helvetica Neue", 9, 'bold' ))
        res = gui_status(rms_id)
        chatWindow.insert(END, "Bot: " + res + '\n\n')
        chatWindow.config(state=DISABLED)
        chatWindow.yview(END)

In [289]:
def quick_help():
    chatWindow.config(state=NORMAL)
    chatWindow.config(foreground="#3b5998", font=("Helvetica Neue", 9, 'bold'))
    res = "Hi there! Here are a few options for your quick help \n ----------------------------------------------------------------------------------------- \n -> Internship Referrals                                    - Type 'Intern'  \n ----------------------------------------------------------------------------------------- \n -> Cannot upload Resume                               - Type 'resume'   \n ----------------------------------------------------------------------------------------- \n -> Cannot access referral page                       - Type 'technical'\n ----------------------------------------------------------------------------------------- \n -> Position not found/No 'refer a friend' link - Type 'job'     \n----------------------------------------------------------------------------------------- \n-> Need 'Referral submission' link                   - Type 'link'     \n----------------------------------------------------------------------------------------- \n-> Bonus not received                                       - Type 'Bonus'    \n----------------------------------------------------------------------------------------- \n-> 'Client' Referrals                                            - Type 'client'   \n ----------------------------------------------------------------------------------------- \n Bot: Have a different query??..please feel free to type your question here"
    chatWindow.insert(END, "Bot: " + res + '\n\n')
    chatWindow.config(state=DISABLED)
    chatWindow.yview(END)

In [290]:
root = Tk()
root.title("TRP_ChatBot BETA")
root.geometry("600x700")
root.resizable(width = FALSE, height = FALSE)

''

In [291]:
#Message header of the chatbot
message = "Referral Program ChatBot"
msg1 = tkinter.Message(root, text = message)
msg1.config(bg="#3b5998", fg="snow", font=('Helvetica Neue', 14, 'bold'), justify = RIGHT , width = "700")
msg1.place(x=0, y=0, height = 32, width = 600)

In [292]:
message = "- Get your referral inquiries answered here -"
msg1 = tkinter.Message(root, text = message)
msg1.config(bg="#3b5998", fg="snow", font=('Helvetica Neue', 8, 'italic'), justify = CENTER , width = "700")
msg1.place(x=0, y=32, height = 18, width = 600)

In [293]:
#Create Chat Window
chatWindow = Text(root, bd=1, bg="grey93",  width="50", height="8", font=("Helvetica Neue", 12), foreground="grey7")
chatWindow.place(x=0,y=50, height=500, width=585)

In [294]:
#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=chatWindow.yview, cursor="clock")
scrollbar.place(x= 585,y = 50, height=500)
chatWindow['yscrollcommand'] = scrollbar.set

### Buttons

In [295]:
#Create Buttons

#login_btn = PhotoImage(file = 'enter2.png')

Button1= Button(root, text = 'Enter ->',  width="12", height=5, bd=0, bg="#4267B2", activebackground="#dfe3ee", 
                foreground='#ffffff',font=("Helvetica Neue", 12,'bold'), command = send, borderwidth = 3)
Button1.place(x=500, y=572, height=128, width = 100)


Button4= Button(root, text="Quick help",width="12", height=5, bd=0, bg="#4267B2", 
                activebackground="#dfe3ee", foreground='#ffffff',font=("Helvetica Neue", 10,'bold'), 
                borderwidth = 2, command = quick_help)
Button4.place(x=0, y=522, height=30, width = 200)


Button2= Button(root, text="Referral status", width="12", height=5, bd=0, bg="#4267B2", 
                activebackground="#dfe3ee", foreground='#ffffff',font=("Helvetica Neue", 10,'bold'), 
                borderwidth = 2, command = status)
Button2.place(x=400, y=522, height=30, width = 200)


Button3= Button(root, text="My bonus history", width="12", height=5, bd=0, bg="#4267B2", 
                activebackground="#dfe3ee", foreground='#ffffff',font=("Helvetica Neue", 10, 'bold'), 
                borderwidth = 2, command = bonus)
Button3.place(x=200, y=522, height=30, width = 200)

TclError: image "pyimage28" doesn't exist

In [ ]:
#Message header for the message window

message2 = "Enter your questions below..."
msg2 = tkinter.Message(root, text = message2)
msg2.config(bg="#8b9dc3", fg="snow", font=('Calibri', 9, 'italic'), justify = LEFT, width = "366")
msg2.place(x=0, y= 552, height= 20, width = 600)

In [ ]:
#Create Message Window

messageWindow = Text(root, bd=0, bg="grey99",width="30", height="4", font=("Helvetica Neue", 10), foreground="grey7")
#messageWindow.insert(END, "Please enter your queries here...")
#messageWindow.delete("0.0",END)
messageWindow.place(x=7, y=579, height=113, width=486)

In [ ]:
#Assigning shortcuts and key-bidings to the buttons

root.bind("<Return>",send)
root.bind("<KP_Enter>",send)

In [ ]:
root.mainloop()